# Bug triage with Deep Learning

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from __future__ import print_function, division

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers

In [5]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [6]:
MAX_SEQUENCE_LENGTH_T = 100 # 40
MAX_SEQUENCE_LENGTH_D = 500 # 200
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

### Parse bugs preproprecessed

In [7]:
# Domain to use
DOMAIN = 'eclipse'
METHOD = 'baseline'
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Glove embeddings
GLOVE_DIR='data/embed'
# Save model
SAVE_PATH = 'baseline_feature@number_of_epochs@epochs_64batch({})'.format(DOMAIN)
SAVE_PATH_FEATURE = 'baseline_feature_@number_of_epochs@epochs_64batch({})'.format(DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [8]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)
evaluation = Evaluation(verbose=0)
experiment = Experiment(baseline, evaluation)

#### Loading bug ids in memory

In [9]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


212512

### Dicionário de títulos e descrições

In [10]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 1min 30s, sys: 3.66 s, total: 1min 34s
Wall time: 1min 32s


### Export the corpus train

In [11]:
if EXTRACT_CORPUS:
    corpus = []
    export_file = open(os.path.join(DIR, 'corpus_train.txt'), 'w')
    for bug_id in tqdm(baseline.bug_set):
        bug = baseline.bug_set[bug_id]
        title = bug['title']
        desc = bug['description']
        export_file.write("{}\n{}\n".format(title, desc))
    export_file.close()

## Geração de batches

# Generating tiple of batches

In [12]:
%%time
experiment.prepare_dataset()

Reading train data
CPU times: user 461 ms, sys: 0 ns, total: 461 ms
Wall time: 460 ms


In [13]:
if 2521 in baseline.bug_set:
    print(baseline.bug_set[2521])

{'issue_id': 2521, 'title_word': array([1036,   95,   11,    8,   95,  213,  101,   20, 3086,   95,    1,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0]), 'bug_status': '0\n', 'delta_ts': '2005-05-10 14:55:51 -0400', 'version': '522\n', 'title': 'selecting window in the window menu does not maximize window gfitic', 'component': '566\n', 'resolution': 'FIXED', 'creation_ts': '2001-10-10 22:38:00 -0400', 'bug_severity': '4\n', 'description_word': array([ 241, 3070,   86,  548,  297,  394,    9,  196,   95,   16,  131,
          8,   17,  213, 1931,  196,   95,    2,   27,   22,  130,  783,
        276,   16,   20, 3484,   23,  610,   11,   28,   44,  563,    2,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,  

### Generating the batch test

In [14]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
train_gen = baseline.siam_gen(baseline.train_data, baseline.dup_sets_train, batch_size, 1)
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, valid_sim = baseline.batch_iterator(baseline.train_data, 
                                                                                          baseline.dup_sets_train, 
                                                                                          batch_size_test, 1)
test_gen = ([valid_input_sample['title'], valid_input_pos['title'], valid_input_neg['title'], 
             valid_input_sample['description'], valid_input_pos['description'], valid_input_neg['description'],
            valid_input_sample['info'], valid_input_pos['info'], valid_input_neg['info']], valid_sim)

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description'].shape[1]

CPU times: user 65.8 ms, sys: 7.74 ms, total: 73.6 ms
Wall time: 72.9 ms


In [15]:
valid_input_sample['title'].shape, valid_input_sample['description'].shape, valid_input_sample['info'].shape, valid_sim.shape

((128, 43), (128, 500), (128, 1682), (128,))

### Validar entrada

In [16]:
%%time 

baseline.display_batch(baseline.train_data, baseline.dup_sets_train, 5)

***Title***: add an option to disable date progress bar on product view
***Title***: make date progress bar optional
***Description***: jira task completion is not working now so my date activity report is completely broken and as result new progress bar is always empty which is confusing and annoying please add an option to hide progress bar from the view that may also make people who don want to show their progress more comfortable
***Description***: the date progress bar is an interesting idea but finding it visually distracting and finding its making it harder to hit the find box quickly can it be made optional perhaps part of my reaction is that have lots of longer running tasks and its not very uplifting to never be able to see it go all green
***similar = 1
########################
***Title***: organization and organization manager deployment outside of osgi environment
***Title***: refactor data collection code to support multiple host types environments
***Description***: curr

## Pre-trained embeddings

Loading pretrained word vectors

### Glove

In [17]:
vocab = baseline.load_vocabulary(os.path.join(DIR, 'vocab_embed.pkl'))
#print(np.random.choice(vocab, 10))
# for token in vocab:
#     print(token)

vocabulary loaded


In [18]:
"Total vocabulary: {}".format(len(vocab))

'Total vocabulary: 113554'

In [19]:
def generating_embed(baseline, GLOVE_DIR, EMBEDDING_DIM):
    embeddings_index = {}
    embed_path = os.path.join(GLOVE_DIR, 'glove.42B.300d.txt')
    f = open(embed_path, 'rb')
    #num_lines = sum(1 for line in open(embed_path, 'rb'))

    vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed.pkl'))
    vocab_size = len(vocab) 

    # Initialize uniform the vector considering the Tanh activation
    embedding_matrix = np.random.uniform(-1.0, 1.0, (vocab_size, EMBEDDING_DIM))
    embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

    loop = tqdm(f)
    loop.set_description("Loading Glove")
    for line in loop:
        tokens = line.split()
        word = tokens[0]
        embeddings_index[word] = np.asarray(tokens[1:], dtype='float32')
        loop.update(1)
    f.close()
    loop.close()

    print('Total %s word vectors in Glove 42B 300d.' % len(embeddings_index))

    loop = tqdm(total=vocab_size)
    loop.set_description('Loading embedding from dataset pretrained')
    i = 0
    for word, embed in vocab.items():
        if word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
        else:
            embedding_matrix[i] = np.asarray(embed, dtype='float32')
        loop.update(1)
        i+=1
    loop.close()
    baseline.embedding_matrix = embedding_matrix

In [ ]:
%%time

generating_embed(baseline, GLOVE_DIR=GLOVE_DIR, EMBEDDING_DIM=EMBEDDING_DIM) # MAX_NB_WORDS=MAX_NB_WORDS

vocabulary loaded


## Experiment

### Training and evaluating for each epoch at same time

#### Auxiliary methods train experiment siamese

In [ ]:
from methods.retrieval import Retrieval
from annoy import AnnoyIndex
import numpy as np

In [ ]:
retrieval = Retrieval()

path = 'data/processed/{}'.format(DOMAIN)
path_buckets = 'data/normalized/{}/{}.csv'.format(DOMAIN, DOMAIN)
path_train = 'data/processed/{}/train.txt'.format(DOMAIN)
path_test = 'data/processed/{}/test.txt'.format(DOMAIN)

experiment.retrieval(retrieval, baseline, number_of_columns_info, DOMAIN)

In [ ]:
# Read and create the test queries duplicate
experiment.create_queries(path_test)

#### Hashing bugs by buckets

In [ ]:
issues_by_buckets = experiment.get_buckets_for_bugs()

## Towards Accurate Duplicate Bug Retrieval Using Deep Learning Techniques

https://github.com/tqtg/DuplicateBugFinder

### Embedding layer

In [ ]:
from keras.constraints import MaxNorm
from keras.initializers import TruncatedNormal, RandomUniform

# Is missing the padding_idx used in pytorch
# https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html
# https://stackoverflow.com/questions/54824768/rnn-model-gru-of-word2vec-to-regression-not-learning
def embedding_layer(embeddings, num_words, embedding_dim, max_sequence_length, trainable):
    embedding_layer = Embedding(num_words,
                                  embedding_dim,
                                  name='embedding_layer',
                                  weights=[embeddings],
                                  embeddings_constraint=MaxNorm(max_value=1, axis=0),
                                  #input_length=max_sequence_length,
                                  input_length=None,
                                  trainable=trainable)
    return embedding_layer

### CNN with filter 3,4,5

In [ ]:
import keras
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D

def cnn_model(embedding_layer, max_sequence_length):

    sequence_input = Input(shape=(max_sequence_length,), name='Feature_BugInput')
    #sequence_input = Input(shape=(None,), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    # best combination filter (3, 4, 5) e 128 e 256
    convs = []
    filter_sizes = [3, 4, 5]
    n_filters = 64

    for index, filter_size in enumerate(filter_sizes):
        l_conv = Conv1D(filters=n_filters, kernel_size=filter_size)(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=filter_size)(l_conv) # index+1
        convs.append(l_pool)

    l_merge = Concatenate(axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    #conv = Conv1D(filters=n_filters * 3, kernel_size=3)(l_merge)
    layer = GlobalAveragePooling1D()(l_merge)
    #layer = Flatten()(l_merge)
    layer = Dense(300, activation='tanh')(layer)
    #layer = LeakyReLU()(layer)

    cnn_feature_model = Model(inputs=[sequence_input], outputs=[layer], name = 'FeatureCNNGenerationModel') # inputs=visible

    return cnn_feature_model

### Bi-LSTM

In [ ]:
from keras.layers import Dense, Input, LSTM, GRU, Dropout, Bidirectional, GlobalAveragePooling1D

def lstm_model(embedding_layer, max_sequence_length):
    number_lstm_units = 50
    rate_drop_lstm = 0
    recurrent_dropout = 0

    sequence_input = Input(shape=(max_sequence_length, ), name='Feature_BugInput')
    #sequence_input = Input(shape=(None, ), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)

    # Creating LSTM Encoder
#     lstm_layer = Bidirectional(LSTM(number_lstm_units, return_sequences=True), # dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm 
#                                merge_mode='ave')

    lstm_layer = LSTM(number_lstm_units, return_sequences=True)(embedded_sequences)
    layer = LSTM(number_lstm_units)(lstm_layer)

    #layer = lstm_layer(embedded_sequences)
    #layer = GlobalAveragePooling1D()(layer)
    layer = Dense(300, activation='tanh')(layer)

    lstm_feature_model = Model(inputs=[sequence_input], outputs=[layer], name = 'FeatureLstmGenerationModel') # inputs=visible

    return lstm_feature_model

### MLP

In [ ]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [ ]:
from keras import backend as K
import tensorflow as tf

def normalize(x, axis):
    norm = K.sqrt(K.sum(K.square(x), axis=axis, keepdims=False))
    return x, K.maximum(norm, K.epsilon())
    
# https://github.com/keras-team/keras/issues/3031
# https://github.com/keras-team/keras/issues/8335
def cosine_distance(inputs):
    x, y = inputs
    x, x_norm = normalize(x, axis=-1)
    y, y_norm = normalize(y, axis=-1)
    distance = K.sum( x * y, axis=-1) / (x_norm * y_norm)
    distance = (distance + K.constant(1)) / K.constant(2)
    # Distance goes from 0 to 2 in theory, but from 0 to 1 if x and y are both
    # positive (which is the case after ReLU activation).
    return K.mean(distance, axis=-1, keepdims=False)
    
class MarginLoss(keras.layers.Layer):
    def call(self, inputs):
        pos = inputs[0]
        neg = inputs[1]
        loss = self.margin_loss(pos, neg)
        self.add_loss(loss, inputs=inputs)
        return inputs
        
    def margin_loss(self, pos, neg):
        margin = K.constant(1.0)
        return K.sum(K.maximum(0.0, margin - pos + neg))

def custom_margin_loss(y_true, y_pred):
    margin = K.constant(1.0)
    pos = y_pred[0]
    neg = y_pred[1]
    return K.sum(K.maximum(0.0, margin - pos + neg))

def pos_distance(y_true, y_pred):
    return y_pred[0]

def neg_distance(y_true, y_pred):
    return y_pred[1]

def stack_tensors(vects):
    return K.stack(vects, axis=-1)

In [ ]:
from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum
from keras.optimizers import Adam, Nadam

def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, name):
  
    bug_t = Input(shape = (sequence_length_t, ), name = 'title_{}'.format(name))
    bug_d = Input(shape = (sequence_length_d, ), name = 'desc_{}'.format(name))
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    
    bug_t_feat = title_feature_model(bug_t)
    bug_d_feat = desc_feature_model(bug_d)
    bug_i_feat = categorical_feature_model(bug_i)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat], name = 'merge_features_{}'.format(name))
    
    #     bug_feature_output = Activation('tanh')(bug_feature_output)
    
    # Bug representation layer
    # bug_feature_output = Dense(300, activation='tanh')(bug_feature_output)
    
    bug_feature_model = Model(inputs=[bug_t, bug_d, bug_i], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [ ]:
def max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=1):
    
    inputs = np.concatenate([encoded_anchor.input, encoded_positive.input, encoded_negative.input], -1).tolist()
    
    encoded_anchor = encoded_anchor.output
    encoded_positive = encoded_positive.output
    encoded_negative = encoded_negative.output
    
    # Cosine
    positive_d = Lambda(cosine_distance, name='pos_cosine_distance', output_shape=[1])([encoded_anchor, encoded_positive])
    negative_d = Lambda(cosine_distance, name='neg_cosine_distance', output_shape=[1])([encoded_anchor, encoded_negative])

    # Loss function only works with a single output
    output = Lambda(
        lambda vects: stack_tensors(vects),
        name='stack-distances',
        output_shape=(2, 1)
    )([positive_d, negative_d])
    
    #loss = MarginLoss()(output)

    similarity_model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

    #optimizer = Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.01)
    optimizer = Adam(lr=1e-3 * decay_lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

    # setup the optimization process 
    similarity_model.compile(optimizer=optimizer, loss=custom_margin_loss, metrics=[pos_distance, neg_distance, custom_margin_loss])

    return similarity_model

In [ ]:
%%time
import keras

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Embeddings
desc_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(vocab), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_D, 
                              trainable=False)
title_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(vocab), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_T, 
                              trainable=False)

# Feature models
'''
    cnn_model
    lstm_model
    mlp_model
'''
desc_feature_model = cnn_model(desc_embedding_layer, MAX_SEQUENCE_LENGTH_D)
title_feature_model = lstm_model(title_embedding_layer, MAX_SEQUENCE_LENGTH_T)
categorical_feature_model = mlp_model(number_of_columns_info)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'in')
encoded_positive = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'pos')

encoded_negative = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'neg')

similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()


'''
    Configuration
'''
epochs = 100
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

'''
    Experiment
'''
for epoch in range(epochs):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, \
            train_sim = baseline.batch_iterator(baseline.train_data, baseline.dup_sets_train, batch_size, 1)
    train_batch = [train_input_sample['title'], train_input_sample['description'], train_input_sample['info'],
                   train_input_pos['title'], train_input_pos['description'], train_input_pos['info'], 
                   train_input_neg['title'], train_input_neg['description'], train_input_neg['info']]
    
#     if epoch == 10:
#         similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=0.1)
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, _ = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets)
        print("Epoch: {} Loss: {:.2f}, MarginLoss: {:.2f}, pos_cosine: {:.2f}, neg_cosine: {:.2f}, recall@25: {:.2f}".format(epoch+1,
                                                                                                         h[0],  h[3],
                                                                                                         h[1], h[2], recall))
    else:
        print("Epoch: {} Loss: {:.2f}, MarginLoss: {:.2f}, pos_cosine: {:.2f}, neg_cosine: {:.2f}".format(epoch+1,
                                                                                                         h[0],  h[3],
                                                                                                         h[1],
                                                                                                         h[2]))
    loss = h[3]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

In [ ]:
_[:20]

In [ ]:
'''
    Between 0-10 epochs recall@25 = 0.28
    Between 0-20 epochs recall@25 = 0.32
    Between 0-70 epochs recall@25 = ?
    Between 0-100 epochs recall@25 = ?
'''
recall, exported_rank = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets)

"recall@25 last epoch:", recall

In [ ]:
# loss=h.history['loss']
# val_loss=h.history['val_loss']

# plt.plot(loss, label='loss')
# plt.plot(val_loss, label='val_loss')
# plt.title('Model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

### Using the feature layers

In [ ]:
def cosine_normalized(a, b): # Cosine used in the siamese model
    a = K.variable(a)
    b = K.variable(b)
    return K.eval(cosine_distance([a, b]))

#### Loading bugs of test

In [ ]:
test = 'eclipse'

In [ ]:
from scipy import spatial
if (DOMAIN == test):
    bug_set = baseline.get_bug_set()
    # Eclipse test
    bug_id = [96204, np.random.choice(list(bug_set))] # non-duplicate {15196, 2}
    # bug_id = [96204, 85581] # duplicate {85581, 96204, 106979}
    dup_a, dup_b = bug_id
    bug_a = bug_set[dup_a]
    bug_b = bug_set[dup_b]

    print(dup_a, dup_b)

#### LSTM feature

In [ ]:
if (DOMAIN == test):
    print(bug_a['title'])
    print(bug_b['title'])

In [ ]:
if (DOMAIN == test):
    bug_vector_a_t = title_feature_model.predict(np.array([bug_a['title_word']]))[0]
    bug_vector_b_t = title_feature_model.predict(np.array([bug_b['title_word']]))[0]
    result = cosine_normalized(bug_vector_a_t, bug_vector_b_t)
    print(result)

In [ ]:
if (DOMAIN == test):
    bug_vector_a_t, bug_vector_b_t

#### CNN feature

In [ ]:
if (DOMAIN == test):
    print(bug_a['description'])
    print(bug_b['description'])

In [ ]:
if (DOMAIN == test):
    bug_vector_a_d = desc_feature_model.predict(np.array([bug_a['description_word']]))[0]
    bug_vector_b_d = desc_feature_model.predict(np.array([bug_b['description_word']]))[0]
    result = cosine_normalized(bug_vector_a_d, bug_vector_b_d)
    print(result)

In [ ]:
if (DOMAIN == test):
    bug_vector_a_d, bug_vector_b_d

#### MLP feature

In [ ]:
if (DOMAIN == test):
    bug_vector_a_i = categorical_feature_model.predict(np.array([retrieval.get_info(bug_a)]))[0]
    bug_vector_b_i = categorical_feature_model.predict(np.array([retrieval.get_info(bug_b)]))[0]
    result = cosine_normalized(bug_vector_a_i, bug_vector_b_i)
    print(result)

In [ ]:
if (DOMAIN == test):
    bug_vector_a_i, bug_vector_b_i

#### Merge features

In [ ]:
if (DOMAIN == test):
    bug_vector_a = np.concatenate([ bug_vector_a_i, bug_vector_a_t, bug_vector_a_d ], -1)
    bug_vector_b = np.concatenate([ bug_vector_b_i, bug_vector_b_t, bug_vector_b_d ], -1)
    result = cosine_normalized(bug_vector_a, bug_vector_b)
    print(result)

In [ ]:
if (DOMAIN == test):
    bug_vector_a, bug_vector_b

### Retrieval evaluation

In [ ]:
print("Total of queries:", len(retrieval.test))

#### Getting the model trained

In [ ]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

In [ ]:
model = experiment.get_model_vectorizer(path=SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)))

In [ ]:
model.summary()

In [ ]:
recall, exported_rank = experiment.evaluate_validation_test(0, model, retrieval.test, issues_by_buckets)

In [ ]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))

In [ ]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [ ]:
'''
# Eclipse
    With CNN print all embeddings zero and 2 epochs
    {'1 - recall_at_5': 0.13,
     '2 - recall_at_10': 0.18,
     '3 - recall_at_15': 0.22,
     '4 - recall_at_20': 0.24}
     Without relu activation for each feature siamese in 100 epochs
     {'1 - recall_at_5': 0.16,
     '2 - recall_at_10': 0.23,
     '3 - recall_at_15': 0.27,
     '4 - recall_at_20': 0.31}
     Without dense in the last layer with 100 epochs with embed trainable
     {'1 - recall_at_5': 0.16,
     '2 - recall_at_10': 0.22,
     '3 - recall_at_15': 0.26,
     '4 - recall_at_20': 0.3}
      
      {'1 - recall_at_5': 0.16,
         '2 - recall_at_10': 0.22,
         '3 - recall_at_15': 0.26,
         '4 - recall_at_20': 0.29,
         '5 - recall_at_25': 0.29}
    With title (100 padding) and desc (500 padding) and batch refactored
        {'1 - recall_at_5': 0.2,
         '2 - recall_at_10': 0.26,
         '3 - recall_at_15': 0.3,
         '4 - recall_at_20': 0.33,
         '5 - recall_at_25': 0.33}
         
         {'1 - recall_at_5': 0.2,
         '2 - recall_at_10': 0.27,
         '3 - recall_at_15': 0.31,
         '4 - recall_at_20': 0.34,
         '5 - recall_at_25': 0.34}
         With recall in validation step and split 90 train 10 to test
         {'1 - recall_at_5': 0.25,
         '2 - recall_at_10': 0.32,
         '3 - recall_at_15': 0.37,
         '4 - recall_at_20': 0.4,
         '5 - recall_at_25': 0.4}
         With 200 epochs validation_recall@25 = 58, optimizer=Nadam
         {'1 - recall_at_5': 0.26,
         '2 - recall_at_10': 0.34,
         '3 - recall_at_15': 0.39,
         '4 - recall_at_20': 0.42,
         '5 - recall_at_25': 0.42}
         With 100 epochs validation_recall@25 = 52, optimizer=Adam
         {'1 - recall_at_5': 0.23,
         '2 - recall_at_10': 0.3,
         '3 - recall_at_15': 0.34,
         '4 - recall_at_20': 0.37,
         '5 - recall_at_25': 0.37}
        With 1000 epochs validation_recall@25=60, optimizer=Nadam
        {'1 - recall_at_5': 0.24,
         '2 - recall_at_10': 0.32,
         '3 - recall_at_15': 0.37,
         '4 - recall_at_20': 0.41,
         '5 - recall_at_25': 0.41}
         With 1000 epochs validation_recall@25=64, optimizer=Nadam
         {'1 - recall_at_5': 0.28,
         '2 - recall_at_10': 0.36,
         '3 - recall_at_15': 0.41,
         '4 - recall_at_20': 0.45,
         '5 - recall_at_25': 0.45}
         Withou change the distance x when calculate the cosine
         {'1 - recall_at_5': 0.18,
         '2 - recall_at_10': 0.24,
         '3 - recall_at_15': 0.28,
         '4 - recall_at_20': 0.31,
         '5 - recall_at_25': 0.31}
         With concatenation
         {'1 - recall_at_5': 0.23,
         '2 - recall_at_10': 0.31,
         '3 - recall_at_15': 0.36,
         '4 - recall_at_20': 0.4,
         '5 - recall_at_25': 0.43}
             
    # Open Office
    {'1 - recall_at_5': 0.2,
     '2 - recall_at_10': 0.27,
     '3 - recall_at_15': 0.31,
     '4 - recall_at_20': 0.34,
     '5 - recall_at_25': 0.34}
'''
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report